### **Generating all possible solutions for [mathler](https://mathler) game** *([Wordle](https://www.nytimes.com/games/wordle/index.html)'s mathematic variant)*

First of all, I thought this wouldn't be feasable, that the ammount of possible equations and results would be to large to compute. But I wanted to try it anyways, and it turns out, there aren't that many valid possible equations.

Let's list the explicit rules for each possible solution:

+ Each equation must be exactly 6 chars long *(each char is considered to be a single digit number or an operator)*
+ Numbers and operators can appear more than once
+ No leading zeros are allowed
+ Calculations follow the order of operations **PEDMAS**
    1. Parentheses
    2. Exponents
    3. Division or Multiplication
    4. Addition or Subtraction

We can also extrapolate some other rules based on the previous requirements:

+ Equations can start with a `'-'` sign, but can't start with any other operator *(`'/','*','+'`)*
+ Equations can't end with any of the operators *(`'/','*','+','-'`)*
+ Only divisions that result in an integer are valid
+ Divisions by zero are not allowed
+ There can't be equations that are equal to the solution:
  + 6 digit numbers
  + 5 digit negative numbers

Besides all of that, there are 3 difficulty levels:

+ **Easy Mathler** *(5 squares and 1 operator)*
+ **Mathler** *(6 squares and 2 operators)*
+ **Hard Mathler** *(8 squares and 3 operators)*
  
#### Let's start with the **Easy Mathler** level:

In [90]:
from tqdm import tqdm
import itertools
import random
import csv
import re
import os

# Import both the solver.py helper script and the constants
from solver import solve
from const import *

# TODO: Get CWD

# Defining constants
LEVELS = {
    "easy": {
        "filepath": "data\\equations_easy.csv",
        "number_of_operators": 1,
        "number_of_chars": 5,
    },
    "int": {
        "filepath": "data\\equations_int.csv",
        "number_of_operators": 2,
        "number_of_chars": 6,
    },
    "hard": {
        "filepath": "data\\equations_hard.csv",
        "number_of_operators": 8,
        "number_of_chars": 3,
    },
}

DIFFICULTY = "easy"
# DIFFICULTY = "int"
# DIFFICULTY = "hard"

EQUATIONS_FILEPATH = LEVELS[DIFFICULTY]["filepath"]
NUMBER_OF_OPERATORS = LEVELS[DIFFICULTY]["number_of_operators"]
NUMBER_OF_CHARS = LEVELS[DIFFICULTY]["number_of_chars"]

CHARS_PATTERN = [NUMBERS + OPERATORS] * NUMBER_OF_CHARS

# Defining helper functions
def print_preview(equations):
    """Print the first and last elements of a list and it's length"""

    print(f"First elements:\t{equations[:5]}")
    print(f"Random elements:{random.choices(equations, k=5)}")
    print(f"Last elements:\t{equations[-5:]}")
    print(f"Length:\t\t{len(equations):,}")


def write_csv(filepath, equations):
    """Writes rows to a csv file"""

    # Remove any previous file, if it exists
    if os.path.exists(filepath):
        os.remove(filepath)

    # Write equations to file
    with open(filepath, "a", newline="") as file:
        writer = csv.writer(file)

        # Iterate over equations and write each row
        for equation in tqdm(equations, desc="Writing equations to csv"):
            writer.writerow(equation)

We will now generate all combinations of elements and convert each of them to a `string`

In [91]:
# List of possible elements for each position of the equation
equations = list(itertools.product(*CHARS_PATTERN))

# Convert equations to strings
equations = list(map(lambda x: "".join(x), tqdm(equations)))

# Print a preview
print_preview(equations)

100%|██████████| 537824/537824 [00:00<00:00, 2444611.84it/s]

First elements:	['11111', '11112', '11113', '11114', '11115']
Random elements:['63681', '46-**', '813/3', '33961', '742/9']
Last elements:	['----0', '----/', '----*', '----+', '-----']
Length:		537,824


Discard equations without the exact number of specified operators.

In [92]:
# Create a temporary list
temp = []

# Iterate over equations
for equation in tqdm(equations):
    counter = 0

    # Iterate over each character in the equation
    for char in equation:

        # If the character is an operator, increment the counter
        if char in OPERATORS:
            counter += 1
    
    # If the first character is a negative operator, decrement the counter
    if equation[0] == NEGATIVE_OPERATORS[0]:
        counter -= 1

    # If the counter is equal to the number of operators, add the equation to the temporary list
    if counter == NUMBER_OF_OPERATORS:
        temp.append(equation)

# Copy the temporary list to the equations list
equations = temp.copy()

# Print a preview
print_preview(equations)

100%|██████████| 537824/537824 [00:00<00:00, 923939.87it/s]

First elements:	['1111/', '1111*', '1111+', '1111-', '1112/']
Random elements:['068+1', '/9891', '703+1', '145/2', '121/8']
Last elements:	['--006', '--007', '--008', '--009', '--000']
Length:		206,000


Discard equations containing consecutive operators.

In [93]:
# Defining the Regex pattern
pattern = r"[/,*,+,-][/,*,+,-]"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

-/111


100%|██████████| 206000/206000 [00:00<00:00, 1089807.49it/s]

First elements:	['1111/', '1111*', '1111+', '1111-', '1112/']
Random elements:['+9834', '484+9', '4/055', '+2852', '7152-']
Last elements:	['-0-06', '-0-07', '-0-08', '-0-09', '-0-00']
Length:		202,000


Discard equations starting with an operator.

In [94]:
# Defining the Regex pattern
pattern = r"^[/,*,+]"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

/1111


100%|██████████| 202000/202000 [00:00<00:00, 1002423.57it/s]

First elements:	['1111/', '1111*', '1111+', '1111-', '1112/']
Random elements:['3440+', '5925/', '51+78', '4*583', '-9*26']
Last elements:	['-0-06', '-0-07', '-0-08', '-0-09', '-0-00']
Length:		172,000


Discard equations ending with an operator.

In [95]:
# Defining the Regex pattern
pattern = r"[/,*,+,-]$"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

1111/


100%|██████████| 172000/172000 [00:00<00:00, 959833.28it/s]

First elements:	['111/1', '111/2', '111/3', '111/4', '111/5']
Random elements:['055/3', '-66-7', '455+9', '492+3', '24+86']
Last elements:	['-0-06', '-0-07', '-0-08', '-0-09', '-0-00']
Length:		128,000


Discard equations containing a single number *(6 digit positive numbers and 5 digit negative numbers)*.

In [96]:
# Defining the Regex pattern
pattern = r"[0-9,-][0-9]{5}"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

100%|██████████| 128000/128000 [00:00<00:00, 1132971.30it/s]

First elements:	['111/1', '111/2', '111/3', '111/4', '111/5']
Random elements:['15*18', '065+5', '179*5', '8*653', '4*688']
Last elements:	['-0-06', '-0-07', '-0-08', '-0-09', '-0-00']
Length:		128,000


Discard equations containing leading zeros.

In [97]:
# Defining the Regex pattern
pattern = r"0\d"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

11/01


100%|██████████| 128000/128000 [00:00<00:00, 946685.30it/s]

First elements:	['111/1', '111/2', '111/3', '111/4', '111/5']
Random elements:['521*7', '685/5', '4*428', '48*91', '-8/59']
Last elements:	['-0-96', '-0-97', '-0-98', '-0-99', '-0-90']
Length:		104,400


Discard equations containing divisions by zero.

In [98]:
# Defining the Regex pattern
pattern = r"/0"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

111/0


100%|██████████| 104400/104400 [00:00<00:00, 1085484.02it/s]

First elements:	['111/1', '111/2', '111/3', '111/4', '111/5']
Random elements:['735-6', '4+123', '897+0', '758/8', '92+98']
Last elements:	['-0-96', '-0-97', '-0-98', '-0-99', '-0-90']
Length:		103,500


Discard equations that have invalid solutions. Then, sort by solution.

In [99]:
# Create a temporary list
temp = []

# Append equations with valid solutions to 'temp' list
for equation in tqdm(equations):
    solution = solve(equation)

    if solution:
        temp.append((equation, solution))

# Sort by solution
equations = sorted(temp, key=lambda x: x[1])

# Print a preview
print_preview(equations)

100%|██████████| 103500/103500 [00:01<00:00, 85037.41it/s]

First elements:	[('0-999', -999), ('1-999', -998), ('0-998', -998), ('1-998', -997), ('2-999', -997)]
Random elements:[('42+57', 99), ('78*19', 1482), ('12-75', -63), ('15-74', -59), ('139+2', 141)]
Last elements:	[('99*97', 9603), ('98*98', 9604), ('98*99', 9702), ('99*98', 9702), ('99*99', 9801)]
Length:		79,145


Check the generated equations against their solutions, making use of Python's built in function `eval()`.

In [100]:
for equation, solution in tqdm(equations):
    if eval(equation) != solution:
        print(f"{equation} = {solution}")
        break

100%|██████████| 79145/79145 [00:00<00:00, 141276.66it/s]


Store the equations and solutions in a `.csv` file.

In [101]:
write_csv(EQUATIONS_FILEPATH, equations)

Writing equations to csv: 100%|██████████| 79145/79145 [00:00<00:00, 297202.44it/s]
